#### Statistical Summary of raw CSVs 🧐
##### Used for PoC

In [2]:
import pandas as pd

In [3]:
header_input_file = r"..\data\bronze_raw\CBP AMS Shipping Data\raw-layer\2018\header\ams_header_2018_combined.csv"
container_input_file = r"..\data\bronze_raw\CBP AMS Shipping Data\raw-layer\2018\container\ams_container_2018_combined.csv"

In [4]:
# sees all columns
pd.set_option('display.max_columns', None)
df_header_test = pd.read_csv(header_input_file, nrows=10)
df_header_test

,identifier,carrier_code,vessel_country_code,vessel_name,port_of_unlading,estimated_arrival_date,foreign_port_of_lading_qualifier,foreign_port_of_lading,manifest_quantity,manifest_unit,weight,weight_unit,measurement,measurement_unit,record_status_indicator,place_of_receipt,port_of_destination,foreign_port_of_destination_qualifier,foreign_port_of_destination,conveyance_id_qualifier,conveyance_id,in_bond_entry_type,mode_of_transportation,secondary_notify_party_1,secondary_notify_party_2,secondary_notify_party_3,secondary_notify_party_4,secondary_notify_party_5,secondary_notify_party_6,secondary_notify_party_7,secondary_notify_party_8,secondary_notify_party_9,secondary_notify_party_10,actual_arrival_date
0,201801010,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",21,CTN,183,Kilograms,0,NaN,New,KAOHSIUNG TAIWAN,NaN,NaN,NaN,IMO Number/Lloyds Number,9300398,NaN,"Vessel, containerized",EGLV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-15
1,201801011,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",3,CAS,1096,Kilograms,0,NaN,New,TAOYUAN TAIWAN,NaN,NaN,NaN,IMO Number/Lloyds Number,9300398,NaN,"Vessel, containerized",EGLV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-15
2,201801012,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",59,CTN,758,Kilograms,0,NaN,New,KAOHSIUNG TAIWAN,NaN,NaN,NaN,IMO Number/Lloyds Number,9300398,NaN,"Vessel, containerized",EGLV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-15
3,201801013,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",168,CTN,2050,Kilograms,0,NaN,New,TAOYUAN TAIWAN,NaN,NaN,NaN,IMO Number/Lloyds Number,9300398,NaN,"Vessel, containerized",EGLV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-15
4,201801014,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",9,CTN,149,Kilograms,0,NaN,New,TAOYUAN TAIWAN,NaN,NaN,NaN,IMO Number/Lloyds Number,9300398,NaN,"Vessel, containerized",EGLV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-15
5,201801015,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",224,CTN,2540,Kilograms,0,NaN,New,TAICHUNG TAIWAN,NaN,NaN,NaN,IMO Number/Lloyds Number,9300398,NaN,"Vessel, containerized",EGLV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-15
6,201801016,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",95,CTN,2173,Kilograms,0,NaN,New,TAICHUNG TAIWAN,NaN,NaN,NaN,IMO Number/Lloyds Number,9300398,NaN,"Vessel, containerized",EGLV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-15
7,201801017,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",349,CTN,5911,Kilograms,0,NaN,New,TAOYUAN TAIWAN,NaN,NaN,NaN,IMO Number/Lloyds Number,9300398,NaN,"Vessel, containerized",EGLV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-15
8,201801018,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",20,CTN,182,Kilograms,0,NaN,New,TAICHUNG TAIWAN,NaN,NaN,NaN,IMO Number/Lloyds Number,9300398,NaN,"Vessel, containerized",EGLV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-15
9,201801019,DFDS,GB,EVER SIGMA,"Los Angeles, California",2017-02-14,Schedule K Foreign Port,"Kaohsiung,China (Taiwan)",2,CTN,17,Kilograms,0,NaN,New,TAICHUNG TAIWAN,NaN,NaN,NaN,IMO Number/Lloyds Number,9300398,NaN,"Vessel, containerized",EGLV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-15


##### Summary of concatenated header CSV (raw)

##### Time took: 1m 8.5s

In [ ]:
unit_counts = {"kg": 0, "lbs": 0, "other": 0}

# for statistical analysis only on numerical columns
count = 0
running_sum_kg = 0.0
running_mean_kg = 0.0
# inf is infinity, first number that appears becomes the min and max
global_min = float("inf")
global_max = float("-inf")

chunk_size = 100000
# what columns are numerical (what we can actually do statisical analysis on)
header_numerical_columns = ["weight"]

# what measurements appear
unit_counts = {"kg": 0, "lbs": 0, "other": 0}

# for statistical analysis only on numerical columns
count = 0
running_sum_kg = 0.0
running_mean_kg = 0.0
# inf is infinity, first number that appears becomes the min and max
global_min = float("inf")
global_max = float("-inf")

for chunk in pd.read_csv(header_input_file, chunksize=chunk_size):
    # NOTE to self: Use vectorization here instead of more for loops, much faster
    # I also don't think the for loops finished running before changing


    # turns weigh unit types to lowercase
    unit_lower = chunk["weight_unit"].astype(str).str.lower()

    # boolean masks
    is_lb = unit_lower.isin(["lb", "pounds"])
    is_kg = unit_lower.isin(["kg", "kilograms"])
    is_other = ~(is_lb | is_kg)

    # count units for types of measurements for weight
    unit_counts["lbs"] += is_lb.sum()
    unit_counts["kg"] += is_kg.sum()
    unit_counts["other"] += is_other.sum()

    # converts weight to kg for stats
    weights_kg = chunk["weight"].copy()
    weights_kg[is_lb] = weights_kg[is_lb] * 0.453592
    # kg and other are left as-is

    # streaming stats
    count += len(weights_kg)
    running_sum_kg += weights_kg.sum()
    global_min = min(global_min, weights_kg.min())
    global_max = max(global_max, weights_kg.max())

C:\Users\stanl\AppData\Local\Temp\ipykernel_15092\181358287.py:26: DtypeWarning: Columns (26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(header_input_file, chunksize=chunk_size):
C:\Users\stanl\AppData\Local\Temp\ipykernel_15092\181358287.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[18979.650056  3956.229424 19999.778464 ... 24969.786008 19959.862368
  3184.669432]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weights_kg[is_lb] = weights_kg[is_lb] * 0.453592
C:\Users\stanl\AppData\Local\Temp\ipykernel_15092\181358287.py:26: DtypeWarning: Columns (26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(header_input_file, chunksize=chunk_size):
C:\Users\stanl\AppData\Local\Temp\ipykernel_15092\181358287.py:43: FutureWarning: 

Unit counts: {'kg': np.int64(18161173), 'lbs': np.int64(1724752), 'other': np.int64(18611)}
Count: 19904536
Sum (kg): 928254111404.1335
Mean (kg): 46635.30520903042
Min (kg): 0.453592
Max (kg): 8703230060.0


C:\Users\stanl\AppData\Local\Temp\ipykernel_15092\181358287.py:26: DtypeWarning: Columns (25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(header_input_file, chunksize=chunk_size):
C:\Users\stanl\AppData\Local\Temp\ipykernel_15092\181358287.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[24687.651784 10509.72664   6350.288    ... 12909.681912 57300.462992
 17979.026104]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weights_kg[is_lb] = weights_kg[is_lb] * 0.453592
C:\Users\stanl\AppData\Local\Temp\ipykernel_15092\181358287.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[11730.796304  2135.964728 20507.347912 ...  1304.984184  1360.776
 11666.38624 ]' has dtype incompatible with int64, please explicitly ca

In [8]:
running_mean_kg = running_sum_kg / count

print("Unit counts:", unit_counts)
print("Count:", count)
print("Sum (kg):", running_sum_kg)
print("Mean (kg):", running_mean_kg)
print("Min (kg):", global_min)
print("Max (kg):", global_max)

Unit counts: {'kg': np.int64(18161173), 'lbs': np.int64(1724752), 'other': np.int64(18611)}
Count: 19904536
Sum (kg): 928254111404.1335
Mean (kg): 46635.30520903042
Min (kg): 0.453592
Max (kg): 8703230060.0


##### Summary of container CSV (raw)

In [7]:
chunk_size = 100000
# what columns are numerical (what we can actually do statisical analysis on)
header_numerical_columns = ["container_length", "container_height", "container_width"]